## Robust fitting of linear models using Least Absolute Deviations

https://en.wikipedia.org/wiki/Least_absolute_deviations

In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["figure.dpi"] = 150
rcParams["savefig.dpi"] = 150
rcParams["text.usetex"] = True

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
np.random.seed(123)

In [ ]:
from lad import lad

Let's create some fake data and add outliers to them:

In [ ]:
x = np.linspace(0, 10, 100)
noise = 10 * np.random.normal(size=len(x))
y = 10 * x + 10 + noise

In [ ]:
mask = np.arange(len(x)) % 5 == 0
y[mask] = np.linspace(6, 3, len(y[mask])) * y[mask]

In [ ]:
plt.plot(x, y, 'ok', alpha=.5)
plt.plot(x, 10 * x + 10, 'k')

In [ ]:
X = np.vstack([x, np.ones(len(x))])

In [ ]:
X.shape

In [ ]:
y.shape

Let's compute the linear least squares model using ``tf.matrix_solve_ls``:

In [ ]:
with tf.Session() as sess:
    X_tensor = tf.convert_to_tensor(X.T, dtype=tf.float64)
    y_tensor = tf.reshape(tf.convert_to_tensor(y, dtype=tf.float64), (-1, 1))
    coeffs = tf.matrix_solve_ls(X_tensor, y_tensor)

In [ ]:
m, b = sess.run(coeffs)

In [ ]:
plt.plot(x, y, 'ok', alpha=.5)
plt.plot(x, 10 * x + 10, 'k')
plt.plot(x, m * x + b, 'r')

Now let's use the function ``lad`` from ``macaw`` to compute the linear model with the
least absolute deviations:

In [ ]:
l1coeffs = lad(X.T, y, yerr=np.sqrt(np.abs(y)), l1_regularizer=10.)

In [ ]:
m_l1, b_l1 = sess.run(l1coeffs)

In [ ]:
plt.plot(x, y, 'ok', alpha=.5)
plt.plot(x, 10 * x + 10, 'k', label="True line")
plt.plot(x, m * x + b, 'r', label="Least squares")
plt.plot(x, m_l1 * x + b_l1, 'cyan', label="Least absolute deviations")
plt.legend()